### ICSigSys4: Speech emotion recognition using voiced speech and local attention
by Bagus Tris Atmaja (bagus@jaist.ac.jp)  

In this paper we compare the following model for speech emotion recognition:
1. Raw speech, using 2 stack Bidirectional LSTM
2. Voiced speech only, using 2 stack Bidirectional LSTM
3. Raw speech, using 2 stack Bidirectional LSTM and attention model
4. Voiced speech, using 2 stack Bidirectional LSTM and attention model

For silence removal, we evaluate the following variations:
- Effect of various minimum sample duration for silence: 0.001, 0.006, 0.01 s
- Effect of various minimum threshold for silence to be removed: 0.001, 0.007, 0.01 %

that lead 0.001 minimum duration and 0.001 of threshold giving the best result.

In [1]:
# import needed module, unimport unneeded moduld
import numpy as np
from tensorflow import set_random_seed
import matplotlib.pyplot as plt
import audiosegment
import sounddevice as sd
from features import *
from helper import *
from sklearn.preprocessing import label_binarize
from scipy import signal

import os
import sys

import wave
import copy

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Embedding, Conv1D, Conv2D, Dropout, MaxPooling2D, Bidirectional
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
np.random.seed(13)
set_random_seed(15)

In [3]:
# dir path for the data
code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = "/media/bagus/data01/dataset/IEMOCAP_full_release/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [4]:
# read already saved pickle data
import pickle
with open(data_path +'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [5]:
# function to calc feature

def calculate_features(frames, freq, options):
    window_sec = 0.2
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [6]:
# read speech signal from variable data2
x_train_speech = np.load('voiced_feat_without_sil_removal.npy')

In [7]:
# read output label
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4936, 4)

In [8]:
#model 1
def speech_model1():
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(100, 34)))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [9]:
model1 = speech_model1()
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1120256   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
Total params: 2,041,348
Trainable params: 2,041,348
Non-trainable params: 0
_________________________________________________________________


In [38]:
# train data
hist = model1.fit(x_train_speech, Y, batch_size=32, epochs=50, verbose=1, shuffle=False,
                 validation_split=0.33)

Train on 3307 samples, validate on 1629 samples
Epoch 1/50
3307/3307 [==============================] - 30s 9ms/step - loss: 0.0582 - acc: 0.9831 - val_loss: 3.8061 - val_acc: 0.5629
Epoch 2/50
3307/3307 [==============================] - 30s 9ms/step - loss: 0.0641 - acc: 0.9825 - val_loss: 3.7052 - val_acc: 0.5574
Epoch 3/50
3307/3307 [==============================] - 30s 9ms/step - loss: 0.0383 - acc: 0.9894 - val_loss: 3.6128 - val_acc: 0.5494
Epoch 4/50
3307/3307 [==============================] - 29s 9ms/step - loss: 0.0659 - acc: 0.9834 - val_loss: 4.1769 - val_acc: 0.5445
Epoch 5/50
3307/3307 [==============================] - 30s 9ms/step - loss: 0.0834 - acc: 0.9791 - val_loss: 3.5950 - val_acc: 0.5568
Epoch 6/50
3307/3307 [==============================] - 29s 9ms/step - loss: 0.0585 - acc: 0.9876 - val_loss: 3.6311 - val_acc: 0.5427
Epoch 7/50
3307/3307 [==============================] - 30s 9ms/step - loss: 0.0573 - acc: 0.9834 - val_loss: 3.7009 - val_acc: 0.5537
Epoch 8

In [39]:
acc1 = hist.history['val_acc']
print(np.mean(acc1), max(acc1))

0.5499693062625244 0.5770411293626636


In [12]:
# doing silence removal (already saved as npy)
# voiced_feat = []
# for ses_mod in data2:
#     x_head = ses_mod['signal']
#     seg = audiosegment.from_numpy_array(x_head, framerate)
#     voice = seg.filter_silence(0.01, 0.1)
#     st_features = calculate_features(voice.to_numpy_array(), framerate, None)
#     st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
#     voiced_feat.append( st_features.T )
#     #print(i)
    
# voiced_feat = np.array(voiced_feat)
# voiced_feat.shape

In [13]:
voiced_feat = np.load('voiced_feat_file_001_001.npy')

In [34]:
# trained model on model2 = voiced speech
hist2 = model1.fit(voiced_feat, Y, batch_size=32, epochs=50, verbose=1, shuffle=False,
                 validation_split=0.33)

Train on 3307 samples, validate on 1629 samples
Epoch 1/50
3307/3307 [==============================] - 53s 16ms/step - loss: 0.0710 - acc: 0.9782 - val_loss: 3.1209 - val_acc: 0.5697
Epoch 2/50
3307/3307 [==============================] - 57s 17ms/step - loss: 0.0919 - acc: 0.9743 - val_loss: 3.3989 - val_acc: 0.5697
Epoch 3/50
3307/3307 [==============================] - 53s 16ms/step - loss: 0.0860 - acc: 0.9737 - val_loss: 3.2052 - val_acc: 0.5820
Epoch 4/50
3307/3307 [==============================] - 57s 17ms/step - loss: 0.0969 - acc: 0.9755 - val_loss: 3.3356 - val_acc: 0.5353
Epoch 5/50
3307/3307 [==============================] - 54s 16ms/step - loss: 0.0892 - acc: 0.9713 - val_loss: 3.0299 - val_acc: 0.5666
Epoch 6/50
3307/3307 [==============================] - 57s 17ms/step - loss: 0.0810 - acc: 0.9767 - val_loss: 2.9335 - val_acc: 0.5684
Epoch 7/50
3307/3307 [==============================] - 53s 16ms/step - loss: 0.0529 - acc: 0.9813 - val_loss: 3.4897 - val_acc: 0.5623


In [35]:
acc2 = hist2.history['val_acc']
print(np.mean(acc2), max(acc2))

0.5646163289661492 0.5887047270641069


In [16]:
# autoencoder attention function for model3
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

def _time_distributed_dense(x, w, b=None, dropout=None,
                           input_dim=None, output_dim=None,
                           timesteps=None, training=None):
    """Apply `y . w + b` for every temporal slice y of x.
    # Arguments
    x: input tensor.
    w: weight matrix.
    b: optional bias vector.
    dropout: wether to apply dropout (same dropout mask
        for every temporal slice of the input).
        input_dim: integer; optional dimensionality of the input.
        output_dim: integer; optional dimensionality of the output.
        timesteps: integer; optional number of timesteps.
        training: training phase tensor or boolean.
        # Returns
            Output tensor.
    """
    if not input_dim:
        input_dim = K.shape(x)[2]
    if not timesteps:
        timesteps = K.shape(x)[1]
    if not output_dim:
        output_dim = K.shape(w)[1]
    if dropout is not None and 0. < dropout < 1.:
        # apply the same dropout pattern at every timestep
        ones = K.ones_like(K.reshape(x[:, 0, :], (-1, input_dim)))
        dropout_matrix = K.dropout(ones, dropout)
        expanded_dropout_matrix = K.repeat(dropout_matrix, timesteps)
        x = K.in_train_phase(x * expanded_dropout_matrix, x, training=training)
    # collapse time dimension and batch dimension together
    x = K.reshape(x, (-1, input_dim))
    x = K.dot(x, w)
    if b is not None:
        x = K.bias_add(x, b)
    # reshape to 3D tensor
    if K.backend() == 'tensorflow':
        x = K.reshape(x, K.stack([-1, timesteps, output_dim]))
        x.set_shape([None, None, output_dim])
    else:
        x = K.reshape(x, (-1, timesteps, output_dim))
    return x
    
class AttentionDecoder(Recurrent):
    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = _time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [17]:
# model3 attention based
def attention_model(optimizer='rmsprop'):
    model = Sequential()
    model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(100, 34)))
    model.add(Bidirectional(AttentionDecoder(128, 128)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

model3 = attention_model()
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 100, 512)          595968    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 256)          1082880   
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               13107712  
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 2052      
Total params: 14,788,612
Trainable params: 14,788,612
Non-trainable params: 0
_________________________________________________________________


In [36]:
# model3: attention decoder
hist3 = model3.fit(x_train_speech, Y, batch_size=32, epochs=50, verbose=1, shuffle = False, 
                 validation_split=0.33)

Train on 3307 samples, validate on 1629 samples
Epoch 1/50
3307/3307 [==============================] - 114s 34ms/step - loss: 0.0942 - acc: 0.9822 - val_loss: 3.5544 - val_acc: 0.6433
Epoch 2/50
3307/3307 [==============================] - 116s 35ms/step - loss: 0.0961 - acc: 0.9831 - val_loss: 3.4767 - val_acc: 0.6476
Epoch 3/50
3307/3307 [==============================] - 116s 35ms/step - loss: 0.1026 - acc: 0.9813 - val_loss: 3.3323 - val_acc: 0.6341
Epoch 4/50
3307/3307 [==============================] - 106s 32ms/step - loss: 0.0698 - acc: 0.9855 - val_loss: 4.0059 - val_acc: 0.6390
Epoch 5/50
3307/3307 [==============================] - 84s 25ms/step - loss: 0.1158 - acc: 0.9837 - val_loss: 4.3316 - val_acc: 0.6298
Epoch 6/50
3307/3307 [==============================] - 81s 24ms/step - loss: 0.1136 - acc: 0.9752 - val_loss: 3.1634 - val_acc: 0.6335
Epoch 7/50
3307/3307 [==============================] - 85s 26ms/step - loss: 0.0865 - acc: 0.9788 - val_loss: 3.1830 - val_acc: 0.6

In [37]:
acc3 = hist3.history['val_acc']
print(np.mean(acc3), max(acc3))

0.6352977286598447 0.6580724367120648


In [30]:
# model4: voiced + attention decoder
hist4 = model3.fit(voiced_feat, Y, batch_size=32, epochs=50, verbose=1, shuffle=False, 
                 validation_split=0.33)

Train on 3307 samples, validate on 1629 samples
Epoch 1/50
3307/3307 [==============================] - 84s 25ms/step - loss: 0.1650 - acc: 0.9568 - val_loss: 2.5425 - val_acc: 0.6869
Epoch 2/50
3307/3307 [==============================] - 83s 25ms/step - loss: 0.1056 - acc: 0.9707 - val_loss: 2.4580 - val_acc: 0.7035
Epoch 3/50
3307/3307 [==============================] - 84s 25ms/step - loss: 0.0909 - acc: 0.9749 - val_loss: 3.1722 - val_acc: 0.6734
Epoch 4/50
3307/3307 [==============================] - 88s 27ms/step - loss: 0.1082 - acc: 0.9779 - val_loss: 2.7613 - val_acc: 0.6826
Epoch 5/50
3307/3307 [==============================] - 84s 25ms/step - loss: 0.0900 - acc: 0.9809 - val_loss: 2.8511 - val_acc: 0.6986
Epoch 6/50
3307/3307 [==============================] - 93s 28ms/step - loss: 0.1118 - acc: 0.9710 - val_loss: 2.7711 - val_acc: 0.6808
Epoch 7/50
3307/3307 [==============================] - 110s 33ms/step - loss: 0.1080 - acc: 0.9755 - val_loss: 2.7550 - val_acc: 0.6746

In [31]:
acc4 = hist4.history['val_acc']
print(np.mean(acc4), max(acc4))

0.6654634744380792 0.7034990790982126


In [40]:
# Effect of minimum sample of noise removal on performance, try minimum duration 0.06 and 0.1 s
# read feature data mindur = 0.06 s
feat_data2 = np.load('voiced_feat_file_006_001.npy')
feat_data2.shape

(4936, 100, 34)

In [41]:
hist5 = model3.fit(feat_data2, Y, batch_size=32, epochs=50, verbose=1, shuffle=False,
                  validation_split=0.33)

Train on 3307 samples, validate on 1629 samples
Epoch 1/50
3307/3307 [==============================] - 70s 21ms/step - loss: 0.1069 - acc: 0.9822 - val_loss: 3.5604 - val_acc: 0.6317
Epoch 2/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.1208 - acc: 0.9767 - val_loss: 3.1562 - val_acc: 0.6384
Epoch 3/50
3307/3307 [==============================] - 70s 21ms/step - loss: 0.0693 - acc: 0.9867 - val_loss: 3.8230 - val_acc: 0.6139
Epoch 4/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.0860 - acc: 0.9855 - val_loss: 3.7134 - val_acc: 0.6440
Epoch 5/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.0738 - acc: 0.9831 - val_loss: 3.3179 - val_acc: 0.6274
Epoch 6/50
3307/3307 [==============================] - 70s 21ms/step - loss: 0.0779 - acc: 0.9834 - val_loss: 3.6616 - val_acc: 0.6255
Epoch 7/50
3307/3307 [==============================] - 75s 23ms/step - loss: 0.0594 - acc: 0.9891 - val_loss: 4.0713 - val_acc: 0.6255


In [42]:
acc5 = hist5.history['val_acc']
print(np.mean(acc5), max(acc5))

0.6208103130451369 0.6451810927314946


In [43]:
# read feature data, mindur = 0.1 s
feat_data3 = np.load('voiced_feat_file_01_001.npy')
feat_data3.shape

(4936, 100, 34)

In [44]:
hist6 = model3.fit(feat_data3, Y, batch_size=32, epochs=50, verbose=1, shuffle=False,
                  validation_split=0.33)

Train on 3307 samples, validate on 1629 samples
Epoch 1/50
3307/3307 [==============================] - 77s 23ms/step - loss: 0.0795 - acc: 0.9858 - val_loss: 3.9436 - val_acc: 0.6225
Epoch 2/50
3307/3307 [==============================] - 76s 23ms/step - loss: 0.0669 - acc: 0.9882 - val_loss: 3.6074 - val_acc: 0.6176
Epoch 3/50
3307/3307 [==============================] - 77s 23ms/step - loss: 0.0846 - acc: 0.9867 - val_loss: 3.9433 - val_acc: 0.6194
Epoch 4/50
3307/3307 [==============================] - 77s 23ms/step - loss: 0.1031 - acc: 0.9873 - val_loss: 4.0210 - val_acc: 0.6090
Epoch 5/50
3307/3307 [==============================] - 77s 23ms/step - loss: 0.1543 - acc: 0.9791 - val_loss: 3.8535 - val_acc: 0.5955
Epoch 6/50
3307/3307 [==============================] - 77s 23ms/step - loss: 0.0719 - acc: 0.9894 - val_loss: 4.1557 - val_acc: 0.6163
Epoch 7/50
3307/3307 [==============================] - 77s 23ms/step - loss: 0.0652 - acc: 0.9891 - val_loss: 3.9556 - val_acc: 0.6163


In [46]:
acc6 = hist6.history['val_acc']
print(np.mean(acc6), max(acc6))

0.6048618784248646 0.6292203805192692


In [47]:
# Now change the threshold for silence removal, use the same minimum duration silence (0.01)
# read feature data, th = 0.007 s
feat_data4 = np.load('voiced_feat_file_001_007.npy')
feat_data4.shape

(4936, 100, 34)

In [48]:
hist7 = model3.fit(feat_data4, Y, batch_size=32, epochs=50, verbose=1, shuffle=False,
                 validation_split=0.33)

Train on 3307 samples, validate on 1629 samples
Epoch 1/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.6967 - acc: 0.8818 - val_loss: 2.9832 - val_acc: 0.5537
Epoch 2/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.4937 - acc: 0.9026 - val_loss: 2.6333 - val_acc: 0.5599
Epoch 3/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.4133 - acc: 0.9138 - val_loss: 2.7801 - val_acc: 0.5820
Epoch 4/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.3315 - acc: 0.9289 - val_loss: 2.8622 - val_acc: 0.5666
Epoch 5/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.2943 - acc: 0.9395 - val_loss: 2.9996 - val_acc: 0.5740
Epoch 6/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.2378 - acc: 0.9483 - val_loss: 3.0869 - val_acc: 0.5648
Epoch 7/50
3307/3307 [==============================] - 69s 21ms/step - loss: 0.3256 - acc: 0.9353 - val_loss: 3.3222 - val_acc: 0.5537


In [50]:
acc7 = hist7.history['val_acc']
print(np.mean(acc7), max(acc7))

0.5647268262232937 0.5868631061086484


In [51]:
# read feature data, th = 0.01 s
feat_data5 = np.load('voiced_feat_file_001_01.npy')
feat_data5.shape

(4810, 100, 34)

In [52]:
# in this case we need to remove some label data because those data below 0.1s threshold (noise dominant)
emo = np.delete(Y, (1061, 1430, 1500, 1552, 1566, 1574, 1575, 1576, 1862, 1863, 1864, 1865, 1868, 1869,
                    1875, 1878, 1880, 1883, 1884, 1886, 1888, 1890, 1892, 1893, 1930, 1931, 1932, 1969,
                    1970, 1971, 1975, 1976, 1977, 1979, 1980, 1981, 1984, 1985, 1986, 1987, 1988, 1989, 
                    1990, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2002, 2003, 2076, 2106, 2110,
                    2177, 2178, 2179, 2180, 2206, 2241, 2242, 2243, 2245, 2246, 2253, 2254, 2262, 2263, 
                    2357, 2358, 2359, 2362, 2368, 2373, 2374, 2418, 2523, 2525, 2526, 2534, 2539, 2542,
                    2549, 2552, 2553, 2554, 2555, 2556, 2561, 2562, 2563, 2564, 2578, 2670, 2671, 2672, 
                    2692, 2694, 2695, 2728, 2733, 2889, 2890, 3034, 3304, 3511, 3524, 3525, 3528, 3655, 
                    3802, 3864, 3930, 4038, 4049, 4051, 4061, 4193, 4241, 4301, 4302, 4307, 4569, 4570), 0)
emo.shape

(4810, 4)

In [53]:
hist8 = model3.fit(feat_data5, emo, batch_size=32, epochs=50, verbose=1, shuffle=False,
                 validation_split=0.33)

Train on 3222 samples, validate on 1588 samples
Epoch 1/50
3222/3222 [==============================] - 68s 21ms/step - loss: 0.8262 - acc: 0.8240 - val_loss: 2.1766 - val_acc: 0.4767
Epoch 2/50
3222/3222 [==============================] - 67s 21ms/step - loss: 0.5702 - acc: 0.8482 - val_loss: 2.2685 - val_acc: 0.4855
Epoch 3/50
3222/3222 [==============================] - 67s 21ms/step - loss: 0.4322 - acc: 0.8908 - val_loss: 2.3117 - val_acc: 0.4887
Epoch 4/50
3222/3222 [==============================] - 67s 21ms/step - loss: 0.3533 - acc: 0.8994 - val_loss: 2.8748 - val_acc: 0.4994
Epoch 5/50
3222/3222 [==============================] - 67s 21ms/step - loss: 0.2992 - acc: 0.9165 - val_loss: 2.5834 - val_acc: 0.5057
Epoch 6/50
3222/3222 [==============================] - 67s 21ms/step - loss: 0.2485 - acc: 0.9286 - val_loss: 3.0098 - val_acc: 0.5113
Epoch 7/50
3222/3222 [==============================] - 67s 21ms/step - loss: 0.2334 - acc: 0.9460 - val_loss: 3.0300 - val_acc: 0.5157


In [54]:
acc8 = hist8.history['val_acc']
print(np.mean(acc8), max(acc8))

0.5022040302537251 0.5188916877324996
